In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import datetime
import string
import json
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Aayush\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Aayush\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Aayush\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
# Define Seed Page
URL = "https://pureportal.coventry.ac.uk/en/organisations/coventry-university/persons/"

# Define profile URL format. This was obtained by manually examining the profile pages
profile_url = "https://pureportal.coventry.ac.uk/en/persons/"

In [5]:
auther_links = {'name':[],'link':[]}
for j in range(20):
    first = requests.get(f'{URL}?page={j}')
    soup = BeautifulSoup(first.text, 'html.parser')
    for i in range(1,50):
        data = soup.find_all("li", class_=f"grid-result-item grid-result-item-{i}")
        for i in data:
            x = i.find_all('span')
            if x[1].text == "School of Computing, Mathematics and Data Sciences":
                link = i.h3.find('a').get('href')
                name = x[0].text
                auther_links['name'].append(name)
                auther_links['link'].append(link)
                research_request = requests.get(link)
                researchs = BeautifulSoup(first.text, 'html.parser')

auther_links

{'name': ['Mohamed Abdelshafy',
  'Reda Al-Bodour',
  'Ali Al Shakarchi',
  'Rachid Anane',
  'Abdullahi Arabo',
  'Farzana Aslam',
  'Kamal Bentahar',
  'Frank Bierbrauer',
  'Will Blewitt',
  'Ray Brown',
  'Nachiketa Chakraborty',
  'Omid Chatrabgoun',
  'Long Chen',
  'Ashima Chopra',
  'Alireza Daneshkhah',
  'Ian Dunwell',
  'Mahmoud Elbattah',
  'Basil Elmasri',
  'Mark Elshaw',
  'Xiang Fei',
  'Abbas Fotouhi',
  'Marwan Fuad',
  'Daniel Goldsmith',
  'Beate Grawemeyer',
  'Daniyal Haider',
  'John Halloran',
  'Manik Hapsara',
  'Qamar Hayat',
  'Tjun Hoh',
  'Darren Imrie'],
 'link': ['https://pureportal.coventry.ac.uk/en/persons/mohamed-abdelshafy',
  'https://pureportal.coventry.ac.uk/en/persons/reda-al-bodour',
  'https://pureportal.coventry.ac.uk/en/persons/ali-al-shakarchi',
  'https://pureportal.coventry.ac.uk/en/persons/rachid-anane',
  'https://pureportal.coventry.ac.uk/en/persons/abdullahi-arabo',
  'https://pureportal.coventry.ac.uk/en/persons/farzana-aslam',
  'htt

In [6]:
def scrape():
    c = 0
    global researchers_links
    researchers_links = {'name':[],'title':[],'link':[],'abstract':[],'date':[]}
    for i in auther_links['link']:
        research_request = requests.get(f"{i}/publications/")
        researchs = BeautifulSoup(research_request.text, 'html.parser')
        data = researchs.find_all("div", class_="result-container")
        for j in data:
            try:
                researchers_links['title'].append(j.h3.text)
                researchers_links['name'].append(list(auther_links['name'])[c])
                url = j.find('a').get('href')
                researchers_links['link'].append(url)
                try:
                    researchers_links['date'].append(j.find("span", class_="date").text)
                except:
                    researchers_links['date'].append("")
                    
                abstract = requests.get(url)
                abstract = BeautifulSoup(abstract.text, 'html.parser')
                    
                try:
                    researchers_links['abstract'].append(abstract.find("div", class_="textblock").text)
                except:
                    researchers_links['abstract'].append("")

            except:
                continue

        c = c+1

In [7]:
%time scrape()

CPU times: total: 12.9 s
Wall time: 9min 19s


In [8]:
import pandas as pd
df = pd.DataFrame(researchers_links)
df.to_csv('database.csv')
df

,name,title,link,abstract,date
0,Mohamed Abdelshafy,Cross-Layer Multipath Multichannel MAC protoco...,https://pureportal.coventry.ac.uk/en/publicati...,Utilising multiple disjoint paths in multiple ...,Jun 2018
1,Mohamed Abdelshafy,Performance evaluation of Receiver Directed Tr...,https://pureportal.coventry.ac.uk/en/publicati...,Utilising multiple channels can increase the w...,4 May 2017
2,Mohamed Abdelshafy,Reliable Multipath Multi-channel Route Migrati...,https://pureportal.coventry.ac.uk/en/publicati...,The route recovery algorithm is a crucial part...,23 Nov 2017
3,Mohamed Abdelshafy,Resisting blackhole attacks on MANETs,https://pureportal.coventry.ac.uk/en/publicati...,MANET routing protocols are designed based on ...,Jan 2016
4,Mohamed Abdelshafy,Dynamic Source Routing under Attacks,https://pureportal.coventry.ac.uk/en/publicati...,MANET routing protocols are designed based on ...,12 Oct 2015
...,...,...,...,...,...
327,Qamar Hayat,Cracking Behavior of Gd2Zr2O7/YSZ Multi-Layere...,https://pureportal.coventry.ac.uk/en/publicati...,A new multi-layered thermal barrier coating sy...,Jan 2023
328,Qamar Hayat,A Review on In Situ Mechanical Testing of Coat...,https://pureportal.coventry.ac.uk/en/publicati...,Real-time evaluation of materials’ mechanical ...,23 Feb 2022
329,Qamar Hayat,High-Entropy Coatings (HEC) for High-Temperatu...,https://pureportal.coventry.ac.uk/en/publicati...,"High-entropy materials (HEM), including alloys...",18 May 2022
330,Qamar Hayat,Multiscale modelling for fusion and fission ma...,https://pureportal.coventry.ac.uk/en/publicati...,The M4F project brings together the fusion and...,Dec 2021


In [2]:
import pandas as pd
preprocess_df = pd.read_csv('database.csv',usecols = ['name','title','link','abstract','date'])
preprocess_df

,name,title,link,abstract,date
0,Mohamed Abdelshafy,Cross-Layer Multipath Multichannel MAC protoco...,https://pureportal.coventry.ac.uk/en/publicati...,Utilising multiple disjoint paths in multiple ...,Jun 2018
1,Mohamed Abdelshafy,Performance evaluation of Receiver Directed Tr...,https://pureportal.coventry.ac.uk/en/publicati...,Utilising multiple channels can increase the w...,4 May 2017
2,Mohamed Abdelshafy,Reliable Multipath Multi-channel Route Migrati...,https://pureportal.coventry.ac.uk/en/publicati...,The route recovery algorithm is a crucial part...,23 Nov 2017
3,Mohamed Abdelshafy,Resisting blackhole attacks on MANETs,https://pureportal.coventry.ac.uk/en/publicati...,MANET routing protocols are designed based on ...,Jan 2016
4,Mohamed Abdelshafy,Dynamic Source Routing under Attacks,https://pureportal.coventry.ac.uk/en/publicati...,MANET routing protocols are designed based on ...,12 Oct 2015
...,...,...,...,...,...
327,Qamar Hayat,Cracking Behavior of Gd2Zr2O7/YSZ Multi-Layere...,https://pureportal.coventry.ac.uk/en/publicati...,A new multi-layered thermal barrier coating sy...,Jan 2023
328,Qamar Hayat,A Review on In Situ Mechanical Testing of Coat...,https://pureportal.coventry.ac.uk/en/publicati...,Real-time evaluation of materials’ mechanical ...,23 Feb 2022
329,Qamar Hayat,High-Entropy Coatings (HEC) for High-Temperatu...,https://pureportal.coventry.ac.uk/en/publicati...,"High-entropy materials (HEM), including alloys...",18 May 2022
330,Qamar Hayat,Multiscale modelling for fusion and fission ma...,https://pureportal.coventry.ac.uk/en/publicati...,The M4F project brings together the fusion and...,Dec 2021


# 2. Indexing Component

In [3]:
scraped_db = pd.read_csv('database.csv').rename(columns={'Unnamed: 0':'sn'}).reset_index(drop=True)
scraped_db.head()

,sn,name,title,link,abstract,date
0,0,Mohamed Abdelshafy,Cross-Layer Multipath Multichannel MAC protoco...,https://pureportal.coventry.ac.uk/en/publicati...,Utilising multiple disjoint paths in multiple ...,Jun 2018
1,1,Mohamed Abdelshafy,Performance evaluation of Receiver Directed Tr...,https://pureportal.coventry.ac.uk/en/publicati...,Utilising multiple channels can increase the w...,4 May 2017
2,2,Mohamed Abdelshafy,Reliable Multipath Multi-channel Route Migrati...,https://pureportal.coventry.ac.uk/en/publicati...,The route recovery algorithm is a crucial part...,23 Nov 2017
3,3,Mohamed Abdelshafy,Resisting blackhole attacks on MANETs,https://pureportal.coventry.ac.uk/en/publicati...,MANET routing protocols are designed based on ...,Jan 2016
4,4,Mohamed Abdelshafy,Dynamic Source Routing under Attacks,https://pureportal.coventry.ac.uk/en/publicati...,MANET routing protocols are designed based on ...,12 Oct 2015


# 2.1 Preprocess Text

In [4]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

stop_words = stopwords.words('english')
df = scraped_db
def preprocess_df(text):
#     text = .apply(lambda x: ' '.join(x.lower() for x in x.split()))
# #     df['title'] = df['title'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
#     ps = PorterStemmer()
#     df['title'] = df['title'].translate(str.maketrans('','',string.punctuation))
#     df['tite'] = df['title'].apply(lambda x: ' '.join([ps.stem(word) for word in x.split()]))
#     df['title'] = df['title'].str.replace('[^\w\s]','')
    text = text.lower()   # Make lowercase
    text = text.translate(str.maketrans('',
                                      '',
                                      string.punctuation))   # Remove punctuation marks
    text = lematize(text)
    return text

def fwpt(word):
    tag = pos_tag([word])[0][1][0].upper()
    hash_tag = {"V": wordnet.VERB, "R": wordnet.ADV,"N": wordnet.NOUN,"J": wordnet.ADJ}         
    return hash_tag.get(tag, wordnet.NOUN)

def lematize(text):
        tkns = nltk.word_tokenize(text)
        ax = ""
        for each in tkns:
            if each not in sw:
                ax += lemmatizer.lemmatize(each, fwpt(each)) + " "
        return ax

In [5]:
processed_db = scraped_db.copy()
lemmatizer = WordNetLemmatizer()
sw = stopwords.words("english")
lemmatizer = WordNetLemmatizer()
def preprocess_df_data(df):
    df.title = df.title.apply(preprocess_df)
    df.name = df.name.str.lower()
    df = df.drop(columns=['name','date'], axis=1)
    return df
    
preprocess_df_datas = preprocess_df_data(processed_db)
# processed_db.head()
preprocess_df_datas

,sn,title,link,abstract
0,0,crosslayer multipath multichannel mac protocol...,https://pureportal.coventry.ac.uk/en/publicati...,Utilising multiple disjoint paths in multiple ...
1,1,performance evaluation receiver direct transmi...,https://pureportal.coventry.ac.uk/en/publicati...,Utilising multiple channels can increase the w...
2,2,reliable multipath multichannel route migratio...,https://pureportal.coventry.ac.uk/en/publicati...,The route recovery algorithm is a crucial part...
3,3,resist blackhole attack manet,https://pureportal.coventry.ac.uk/en/publicati...,MANET routing protocols are designed based on ...
4,4,dynamic source rout attack,https://pureportal.coventry.ac.uk/en/publicati...,MANET routing protocols are designed based on ...
...,...,...,...,...
327,327,crack behavior gd2zr2o7ysz multilayered therma...,https://pureportal.coventry.ac.uk/en/publicati...,A new multi-layered thermal barrier coating sy...
328,328,review situ mechanical test coating,https://pureportal.coventry.ac.uk/en/publicati...,Real-time evaluation of materials’ mechanical ...
329,329,highentropy coating hec hightemperature applic...,https://pureportal.coventry.ac.uk/en/publicati...,"High-entropy materials (HEM), including alloys..."
330,330,multiscale model fusion fission material m4f p...,https://pureportal.coventry.ac.uk/en/publicati...,The M4F project brings together the fusion and...


# 2.2 Index Construction

In [6]:
single = scraped_db.loc[0,:].copy()
print(single)
indexing_trial = {}

words = single.title.split()
sn = single.sn
word = words[0]
example = {word: [sn]}

print('=====================================================================')
print('Sample index')
print(example)

sn                                                          0
name                                       Mohamed Abdelshafy
title       Cross-Layer Multipath Multichannel MAC protoco...
link        https://pureportal.coventry.ac.uk/en/publicati...
abstract    Utilising multiple disjoint paths in multiple ...
date                                                 Jun 2018
Name: 0, dtype: object
Sample index
{'Cross-Layer': [0]}


In [7]:
## Indexer Function
def apply_index(inputs, index):
    words = inputs.title.split()
    sn = int(inputs.sn)
    for word in words:
        if word in index.keys():
            if sn not in index[word]:
                index[word].append(sn)
        else:
            index[word] = [sn]
    return index

indx = apply_index(inputs=single, index= {})
print(indx)

{'Cross-Layer': [0], 'Multipath': [0], 'Multichannel': [0], 'MAC': [0], 'protocol': [0], 'for': [0], 'MANETs': [0]}


In [8]:
def full_index(df, index):
    for x in range(len(df)):
        inpt = df.loc[x,:]
#         print(inpt)
        ind = apply_index(inputs=inpt, index=index)
    return ind

def construct_index(df, index):
    queue = preprocess_df_data(df)
    ind = full_index(df=queue, index=index)
    return ind

indexed = full_index(preprocess_df_datas, 
                     index = {})


indexes = construct_index(df=scraped_db, 
                          index = {})


In [9]:
indexed

{'crosslayer': [0],
 'multipath': [0, 2],
 'multichannel': [0, 2, 92],
 'mac': [0],
 'protocol': [0, 1, 6, 9, 10, 11, 25],
 'manet': [0, 1, 3, 6],
 'performance': [1, 5, 6, 15, 101, 127, 265, 308],
 'evaluation': [1, 9, 16, 17, 19, 71, 83, 104, 127, 153, 182, 227, 274, 304],
 'receiver': [1],
 'direct': [1],
 'transmission': [1, 220, 263],
 'single': [1],
 'transceiver': [1],
 'reliable': [2],
 'route': [2],
 'migration': [2],
 'multi': [2],
 'linkfailure': [2],
 'wireless': [2, 9, 218, 219, 220, 263, 265, 266, 267, 268, 269, 318],
 'ad': [2],
 'hoc': [2],
 'network': [2,
  23,
  60,
  68,
  92,
  104,
  105,
  113,
  114,
  117,
  128,
  191,
  200,
  203,
  204,
  209,
  212,
  215,
  218,
  219,
  263,
  266,
  267,
  316,
  318],
 'resist': [3, 7],
 'blackhole': [3],
 'attack': [3, 4, 5, 6, 7, 8, 10, 11],
 'dynamic': [4, 29, 89, 146, 214],
 'source': [4, 73, 320],
 'rout': [4, 6, 8],
 'aodv': [5, 6, 7, 8],
 'saodv': [5],
 'comparison': [5, 46],
 'analysis': [6, 8, 9, 15, 62, 73, 10

In [10]:
with open('indexes.json', 'w') as new_f:
    
    json.dump(indexes, new_f, sort_keys=True)
    
with open('indexes.json', 'r') as file:
    data = json.load(file)

def index_2(df, x_path):
    if len(df) > 0:
        with open(x_path, 'r') as file:
            prior_index = json.load(file)
        new_index = construct_index(df = df, index = prior_index)
        with open(x_path, 'w') as new_f:
            json.dump(new_index, new_f, sort_keys=True, indent=4)

# 3. Query Processor

In [11]:
def demonstrate_query_processing():
    sample = input('Enter Search Terms: ')
    processed_query = preprocess_df(sample)
    #print(f'User Search Query: {sample}')
    print(f'Processed Search Query: {processed_query}')
    return processed_query

# 3.1. Split Query into individual terms

In [12]:
def split_query(terms):
    each = preprocess_df(terms)
    return each.split()

dqp = demonstrate_query_processing()
print(f'Split Query: {split_query(dqp)}')

Enter Search Terms: database
Processed Search Query: database 
Split Query: ['database']


## 3.2. Boolean Functionalities

In [13]:
def union(lists):
    union = list(set.union(*map(set, lists)))
    union.sort()
    return union

def intersection(lists):
    intersect = list(set.intersection(*map(set, lists)))
    intersect.sort()
    return intersect

# 3.3. Search Engine Function

In [14]:
def vertical_search_engine(df, query, index=indexes):
    query_split = split_query(query)
    retrieved = []
    for word in query_split:
        if word in index.keys():
            retrieved.append(index[word])
            
            
    # Ranked Retrieval
    if len(retrieved)>0:
        high_rank_result = intersection(retrieved)
        low_rank_result = union(retrieved) 
        c = [x for x in low_rank_result if x not in high_rank_result]      
        high_rank_result.extend(c)
        result = high_rank_result
        final_output = df[df.sn.isin(result)].reset_index(drop=True)
    
        # Return result in order of Intersection ----> Union
        dummy = pd.Series(result, name = 'sn').to_frame()
        result = pd.merge(dummy, final_output, on='sn', how = 'left')
        
    else:
        result = 'No result found'
    
    return result


In [15]:
def test_search_engine():
    xtest = scraped_db.copy()
    query = input("Enter your search query: ")
    return vertical_search_engine(xtest, query, indexed)
    
test_search_engine()

Enter your search query: database for AI


,sn,name,title,link,abstract,date
0,45,rachid anane,hisql frontend query system historical database,https://pureportal.coventry.ac.uk/en/publicati...,The paper describes a prototype system called ...,Nov 1997
1,254,marwan fuad,speedingup similarity search time series datab...,https://pureportal.coventry.ac.uk/en/publicati...,In this paper we present a new generic frame t...,11 Nov 2010
2,272,beate grawemeyer,social explainability ai impact nontechnical e...,https://pureportal.coventry.ac.uk/en/publicati...,"In striving for explainable AI, it is not nece...",23 Jul 2022
3,307,beate grawemeyer,graphical data display database query help use...,https://pureportal.coventry.ac.uk/en/publicati...,This paper describes the process by which we h...,1 Jan 2005
4,308,beate grawemeyer,effect knowledgeofexternalrepresentations upon...,https://pureportal.coventry.ac.uk/en/publicati...,This study examined the representation selecti...,1 Jan 2004


## 4. Schedule Crawler for every week


In [16]:
days = 0
interval = 7
while days <= 1:
    scrape()
    print(f"Crawled at {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f'Next crawl scheduled after {interval} days')
    time.sleep(interval)
    days = days + 1

In [17]:
from tkinter import *
from PIL import Image, ImageTk 
from tkinter import scrolledtext
from pandastable import Table, TableModel
from contextlib import suppress
import warnings
warnings.filterwarnings('ignore')

def new_gui():
    window = Tk()
    window.title("Coventry University")
    window.geometry('1100x1100')
    
    lbl = Label(window, text="Search Engine", font=("Arial Bold", 30), padx=5, pady=5)
    lbl.grid(column=1, row=0)
    
    lbl2 = Label(window, text="Please Enter your search query", font=("Arial", 15), padx=5, pady=5)
    lbl2.grid(column=0, row=1)
    
    
    
    query = Entry(window,width=200)
    query.grid(column=1, row=1,  padx=5, pady=5)
    
    results = Canvas(window, height=1000, width=1000)
    results.grid(column=1, row=2, padx=5, pady=5)
    
    # Entry
    def getInputBoxValue():
        userInput = query.get()
        return userInput
    
    # Button
    def clicked():
        search()
        #pass
        
    def no_result():
        messagebox.showwarning("Warning", "No results found. Please try different search terms")
        
    def search():
        xtest = scraped_db.copy()
        q = query.get()
        f = Frame(window,width=1000)
        df = vertical_search_engine(xtest, q, indexed)
        if type(df) == str:
            no_result()
        else:
            pt = Table(results)
            try:
                table = pt = Table(results, dataframe=df, width = 1000)
                pt.show()
            except AttributeError:
                pass
            
    def close_window():
        if messagebox.askokcancel("Quit", "Quit Programme?"):
            window.destroy()
        
    
    btn = Button(window, text="Search", command=clicked)
    btn.grid(column=2, row=1)
    
    
    window.protocol("WM_DELETE_WINDOW", close_window)       
    window.mainloop()

In [ ]:
new_gui()